In [8]:
import mediapipe as mp
import numpy as np
import cv2
import tensorflow as tf  # For loading your saved .keras model
from sklearn.preprocessing import LabelEncoder

# Load the trained model
model = tf.keras.models.load_model("yoga_pose_model.keras")

# Load the label encoder (you must save it during training)
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('label_encoder_classes.npy')  # Load label encoder classes

# Alias for MediaPipe
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# Set up PoseLandmarker
model_path = 'pose_landmarker_lite.task'
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE
)

# Load image for prediction
image_path = "C:/Users/HP/Desktop/SEM 6/Minor project 2/Book-Recommendation-System-master/Yoga Pose Detector/TRAIN/Veerabhadrasana/Images/Veerabhadrasana_4.jpg"  # Your input image path
test_img = cv2.imread(image_path)


# Ensure the image is valid
if test_img is None:
    print(f"Skipping invalid image: {image_path}")
    exit()

# Convert to RGB
image_rgb = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)

# Convert to MediaPipe Image object
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image_rgb)

with PoseLandmarker.create_from_options(options) as landmarker:
    result = landmarker.detect(mp_image)

    # If landmarks are detected
    if result.pose_landmarks:
        landmarks = result.pose_landmarks[0]  # Assuming single person in the image

        # Flatten the landmarks and prepare feature vector for model
        feature_vector = []
        for lm in landmarks:
            feature_vector.extend([lm.x, lm.y, lm.z, lm.visibility])

        # Convert feature vector to numpy array for prediction
        X = np.array(feature_vector).reshape(1, -1)

        # Predict the pose
        predicted_pose = model.predict(X)
        predicted_class = np.argmax(predicted_pose)  # Get the class index
        pose_name = label_encoder.inverse_transform([predicted_class])[0]  # Decode the class index to pose name

        # Draw landmarks on the image
        for lm in landmarks:
            x_px = int(lm.x * test_img.shape[1])
            y_px = int(lm.y * test_img.shape[0])
            cv2.circle(test_img, (x_px, y_px), 4, (0, 255, 0), -1)  # Green markers

        # Write the detected pose on the image
        cv2.putText(test_img, pose_name, (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

        # Display the image with markers and detected pose
        cv2.imshow("Yoga Pose Detection", test_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    else:
        print("No pose landmarks detected.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


In [7]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [2]:
model=tf.keras.models.load_model('yoga_pose_model.keras')
model.summary()

c:\Users\HP\anaconda3\envs\pose_env\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │        17,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,602 (201.57 KB)

 Trainable params: 25,800 (100.78 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 25,802 (100.79 KB)

In [3]:
mediapipe_model_path='pose_landmarker_lite.task'

In [4]:
BaseOptions = mp.tasks.BaseOptions
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions
PoseLandmarkerResult = mp.tasks.vision.PoseLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode
base_options = python.BaseOptions(model_asset_path=mediapipe_model_path)
KEY_LANDMARKS = [
    mp.solutions.pose.PoseLandmark.NOSE,
    mp.solutions.pose.PoseLandmark.LEFT_SHOULDER,
    mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER,
    mp.solutions.pose.PoseLandmark.LEFT_ELBOW,
    mp.solutions.pose.PoseLandmark.RIGHT_ELBOW,
    mp.solutions.pose.PoseLandmark.LEFT_HIP,
    mp.solutions.pose.PoseLandmark.RIGHT_HIP
]

POSE_CLASSES = {
    0: "ArdhaChandrasana",
    1: "BaddhaKonasana",
    2: "Downward_dog",
    4:"Natarajasana",
    5:"Triangle",
    6:"UtkataKonasana",
    7:"Veerabhadrasana",
}


# Create a pose landmarker instance with the live stream mode:
def process_result(result: PoseLandmarkerResult, image: mp.Image, timestamp_ms: int):
    image_array = image.numpy_view()
    frame = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)


    if result.pose_landmarks:# If the model sent landmarks
        landmarks=[]
        for lm in KEY_LANDMARKS: #Extract only key Landmarks
            landmark = result.pose_landmarks[0][lm.value]
            landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            # Draw the landmark
            h, w = image.height, image.width
            cx, cy = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED)

        landmarks=np.array(landmarks).reshape(1,-1)
        prediction=model.predict(landmarks)

        predicted_class = np.argmax(prediction)
        confidence = np.max(prediction)

        if confidence > 0.7:
            cv2.putText(frame,
                       f"{POSE_CLASSES.get(predicted_class, 'Unknown')}",
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame,
                       f"Confidence: {confidence:.2f}",
                       (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow('Yoga Pose Detection', frame)
    cv2.waitKey(1)





In [5]:
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.LIVE_STREAM,
    result_callback=process_result)

In [ ]:
import mediapipe as mp
import cv2
import time

# Initialize MediaPipe Vision and PoseLandmarker
BaseOptions = mp.tasks.BaseOptions
VisionRunningMode = mp.tasks.vision.RunningMode
PoseLandmarker = mp.tasks.vision.PoseLandmarker
PoseLandmarkerOptions = mp.tasks.vision.PoseLandmarkerOptions

# Path to model and image
model_path = "pose_landmarker_lite.task"  # Make sure this file is in your working directory
image_path = "C:/Users/HP/OneDrive/Documents/SEM5/PA LAB/yoga/tree.jpg"  # Replace with your image filename

# Load the image using OpenCV and convert to RGB
bgr_image = cv2.imread(image_path)
rgb_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2RGB)

# Create a MediaPipe Image object
mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_image)

# Set up the PoseLandmarker
options = PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE,
    output_segmentation_masks=False
)

# Load and run the landmarker
with PoseLandmarker.create_from_options(options) as landmarker:
    result = landmarker.detect(mp_image)

    # Draw landmarks
    if result.pose_landmarks:
        for landmark in result.pose_landmarks[0]:
            x_px = int(landmark.x * bgr_image.shape[1])
            y_px = int(landmark.y * bgr_image.shape[0])
            cv2.circle(bgr_image, (x_px, y_px), 4, (0, 255, 0), -1)

    # Show image with landmarks
    cv2.imshow("Pose Landmarks", bgr_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:

# Main loop
if __name__ == "__main__":
    with vision.PoseLandmarker.create_from_options(options) as landmarker:
        cap = cv2.VideoCapture(0)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
            timestamp_ms = int(cv2.getTickCount() / cv2.getTickFrequency() * 1000)
            landmarker.detect_async(mp_image, timestamp_ms)

            if cv2.waitKey(5) & 0xFF == 27:  # ESC to exit
                break

        cap.release()
        cv2.destroyAllWindows()